In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc 
import sympy as sy 

# Traffic Pattern

使用流量矩阵$\lambda$来描述：（个人认为类似于连续马尔可夫过程的转移矩阵的感觉）


In [3]:
N = 5

## 随机流量：到各个节点的流量都相同

$\lambda_{sd}=1/N$

In [33]:
L = np.ones((N, N)) * (1 / (N - 1)) 
print(L)

[[0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]]


## 排列流量：每一个节点发送给固定的节点

d = $\pi (s)$

In [6]:
import numpy as np

# 定义节点数量
num_nodes = 5

# 创建一个全零矩阵
traffic_matrix = np.zeros((num_nodes, num_nodes))

# 定义排列函数，每个源节点将所有流量发送到指定的节点
permutation = [0, 3, 2, 4, 1]

for i in range(num_nodes):
    traffic_matrix[i, permutation[i]] = 1

print(traffic_matrix)


[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]


In [8]:
import numpy as np

# 定义源地址
source = np.array([0, 1, 2, 3])

# 定义位排列函数
def bit_permutation(source, f, g):
    return np.bitwise_xor(f(source), g(source))

# 定义位补码函数
def bit_complement(source):
    return ~source

# 定义位反转函数
def bit_reverse(source, b):
    return source[::-1]

# 定义位旋转函数
def bit_rotation(source, b):
    return np.mod(source + 1, b)

# 定义洗牌函数
def shuffle(source, b):
    return np.mod(source - 1, b)

# 定义转置函数
def transpose(source, b):
    return np.mod(source + b//2, b)

# 测试这些函数
print("Bit permutation: ", bit_permutation(source, lambda x: x, lambda x: x))
print("Bit complement: ", bit_complement(source))
print("Bit reverse: ", bit_reverse(source, 4))
print("Bit rotation: ", bit_rotation(source, 4))
print("Shuffle: ", shuffle(source, 4))
print("Transpose: ", transpose(source, 4))


Bit permutation:  [0 0 0 0]
Bit complement:  [-1 -2 -3 -4]
Bit reverse:  [3 2 1 0]
Bit rotation:  [1 2 3 0]
Shuffle:  [3 0 1 2]
Transpose:  [2 3 0 1]


In [50]:
import math
import random
import cvxpy as cp
# 节点数量
scale = 8
n = scale * scale  # 假设我们有16个节点，这样就可以构造一个4x4的2D-Torus网络

# 计算网络的尺寸
size = scale

# 构造2D-Torus网络的边
e = []
for i in range(size):
    for j in range(size):
        loc = lambda i, j : i * scale + j
        left = loc((i - 1 + size) % size, j)
        right = loc((i + 1) % size, j)
        up = loc(i, (j + 1) % size)
        down = loc(i, (j - 1 + size) % size)

        # left = loc(i - 1, j) if i - 1 >= 0 else None
        # right = loc(i + 1, j) if i + 1 < size else None 
        # up = loc(i, j - 1) if j - 1 >= 0 else None 
        # down = loc(i, j + 1) if j + 1 < size else None
        this = loc(i, j)
        to = [left, right, up, down]
        for y in to:
            if y is None:
                continue
            e.append((this, y, 1.0))

# 初始化流量矩阵
L = np.ones((size, size)) * (1 / (size - 1))

# 构造流量模式
c = []
for i in range(size):
    for j in range(size):
        if i != j:
            # 每个元素是一个元组，包含源节点、目标节点和需求量
            c.append((i, j, L[i][j]))


print("节点数量：", size)
print("2D-Torus网络的边：", e)
print("流量模式：", c)

def get_result(n, e, c):
    k = len(c)
    m = len(e)
    # 初始化变量
    f = [[[None for _ in range(n)] for _ in range(n)] for _ in range(k)]
    for u, v, w in e:
        for i in range(k):
            f[i][u][v] = cp.Variable(nonneg=True)
    throughput = cp.Variable(nonneg=True)

    def adv_sum(expr):
        return sum([x for x in expr if x is not None])
    # 定义约束
    constraints = []

    # 容量限制
    real_capacity = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(len(e)):
        u, v, capacity = e[i]
        real_capacity[u][v] = capacity
    for i in range(n):
        for j in range(n):
            constraints.append(adv_sum([f[x][i][j] for x in range(k)]) <= real_capacity[i][j])

    # 流守恒
    for i in range(k):
        for u in range(n):
            s, t, d = c[i]
            if u != s and u != t:
                constraints.append(adv_sum([f[i][u][v] for v in range(n)]) - adv_sum(f[i][v][u] for v in range(n)) == 0)

    # 需求的满足
    for i in range(k):
        s, t, d = c[i]
        constraints.append(adv_sum([f[i][s][v] for v in range(n)]) == d * throughput)
        constraints.append(adv_sum([f[i][v][t] for v in range(n)]) == d * throughput)

    # 定义目标函数()
    obj = cp.Maximize(throughput)

    # 构造问题
    prob = cp.Problem(obj, constraints)

    # 求解问题
    prob.solve(solver=cp.ECOS)

    return prob.status, throughput.value

get_result(n, e, c)


节点数量： 8
2D-Torus网络的边： [(0, 56, 1.0), (0, 8, 1.0), (0, 1, 1.0), (0, 7, 1.0), (1, 57, 1.0), (1, 9, 1.0), (1, 2, 1.0), (1, 0, 1.0), (2, 58, 1.0), (2, 10, 1.0), (2, 3, 1.0), (2, 1, 1.0), (3, 59, 1.0), (3, 11, 1.0), (3, 4, 1.0), (3, 2, 1.0), (4, 60, 1.0), (4, 12, 1.0), (4, 5, 1.0), (4, 3, 1.0), (5, 61, 1.0), (5, 13, 1.0), (5, 6, 1.0), (5, 4, 1.0), (6, 62, 1.0), (6, 14, 1.0), (6, 7, 1.0), (6, 5, 1.0), (7, 63, 1.0), (7, 15, 1.0), (7, 0, 1.0), (7, 6, 1.0), (8, 0, 1.0), (8, 16, 1.0), (8, 9, 1.0), (8, 15, 1.0), (9, 1, 1.0), (9, 17, 1.0), (9, 10, 1.0), (9, 8, 1.0), (10, 2, 1.0), (10, 18, 1.0), (10, 11, 1.0), (10, 9, 1.0), (11, 3, 1.0), (11, 19, 1.0), (11, 12, 1.0), (11, 10, 1.0), (12, 4, 1.0), (12, 20, 1.0), (12, 13, 1.0), (12, 11, 1.0), (13, 5, 1.0), (13, 21, 1.0), (13, 14, 1.0), (13, 12, 1.0), (14, 6, 1.0), (14, 22, 1.0), (14, 15, 1.0), (14, 13, 1.0), (15, 7, 1.0), (15, 23, 1.0), (15, 8, 1.0), (15, 14, 1.0), (16, 8, 1.0), (16, 24, 1.0), (16, 17, 1.0), (16, 23, 1.0), (17, 9, 1.0), (17, 25, 1.0),

('optimal', 3.499999999997071)